In [1]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from keras.layers import LSTM

<h1>Read Datasets<h1>

In [2]:
df = pd.read_excel('Datasets_Ready.xlsx')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 0 to 4509
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      4510 non-null   int64  
 1   Text            4510 non-null   object 
 2   emoji           4510 non-null   object 
 3   Tokenized       4510 non-null   object 
 4   final_text      4494 non-null   object 
 5   text_emoji      4510 non-null   object 
 6   Pos_Word        4510 non-null   int64  
 7   Neg_Word        4510 non-null   int64  
 8   Total_Word      4510 non-null   int64  
 9   Pos_Ratio       4510 non-null   float64
 10  Neg_Ratio       4510 non-null   float64
 11  Sentimen_Text   4510 non-null   object 
 12  Tokenize_Emoji  4510 non-null   object 
 13  Pos_Emoji       4510 non-null   int64  
 14  Neg_Emoji       4510 non-null   int64  
 15  Sentimen_Emoji  4510 non-null   object 
 16  Sarcasm         4510 non-null   object 
dtypes: float64(2), int64(6), object(9

,Unnamed: 0,Text,emoji,Tokenized,final_text,text_emoji,Pos_Word,Neg_Word,Total_Word,Pos_Ratio,Neg_Ratio,Sentimen_Text,Tokenize_Emoji,Pos_Emoji,Neg_Emoji,Sentimen_Emoji,Sarcasm
0,0,kawan2 sunda bersatu jemput arteria dahlan,😁,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",kawan sunda bersatu jemput arteria dahlan,kawan sunda bersatu jemput arteria dahlan 😁,2,0,6,0.333333,0.000000,Positif,"['kawan', 'sunda', 'bersatu', 'jemput', 'arter...",1,0,Positif,Negatif
1,2,arteria dahlan disidang adat sunda daerah menu...,😁,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",arteria dahlan sidang adat sunda daerah menunt...,arteria dahlan sidang adat sunda daerah menunt...,5,4,9,0.555556,0.444444,Positif,"['arteria', 'dahlan', 'sidang', 'adat', 'sunda...",1,0,Positif,Negatif
2,4,ga ditafsirkan membanding bandingkan edy arter...,😁,"['ga', 'tafsir', 'membanding', 'membandingkan'...",ga tafsir membanding membandingkan edy arteria...,ga tafsir membanding membandingkan edy arteria...,3,2,9,0.333333,0.222222,Positif,"['ga', 'tafsir', 'membanding', 'membandingkan'...",1,0,Positif,Negatif
3,5,nantik ngak divhumas polri nya bilang palsu td...,😁,"['nanti', 'tidak', 'divisi humas', 'polri', '....",divisi humas polri bilang palsu mengeluarkan a...,divisi humas polri bilang palsu mengeluarkan a...,4,8,16,0.250000,0.500000,Negatif,"['divisi', 'humas', 'polri', 'bilang', 'palsu'...",1,0,Positif,Positif
4,6,urut dada dech kuasa hukum tdk mengerti hukum ...,😁,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'tid...",urut dada deh kuasa hukum mengerti hukum arter...,urut dada deh kuasa hukum mengerti hukum arter...,14,6,28,0.500000,0.214286,Positif,"['urut', 'dada', 'deh', 'kuasa', 'hukum', 'men...",1,0,Positif,Negatif


In [3]:
# Remove unused column/feature
df = df.drop(columns='Unnamed: 0')
df = df.drop(columns='Text')
df = df.drop(columns='emoji')
df = df.drop(columns='Tokenized')
df = df.drop(columns='final_text')
df = df.drop(columns='Pos_Word')
df = df.drop(columns='Neg_Word')
df = df.drop(columns='Total_Word')
df = df.drop(columns='Pos_Ratio')
df = df.drop(columns='Neg_Ratio')
df = df.drop(columns='Tokenize_Emoji')
df = df.drop(columns='Pos_Emoji')
df = df.drop(columns='Neg_Emoji')
df = df.drop(columns='Sentimen_Emoji')

In [4]:
df.head()

,text_emoji,Sentimen_Text,Sarcasm
0,kawan sunda bersatu jemput arteria dahlan 😁,Positif,Negatif
1,arteria dahlan sidang adat sunda daerah menunt...,Positif,Negatif
2,ga tafsir membanding membandingkan edy arteria...,Positif,Negatif
3,divisi humas polri bilang palsu mengeluarkan a...,Negatif,Positif
4,urut dada deh kuasa hukum mengerti hukum arter...,Positif,Negatif


In [5]:
print(f'Total Duplicated Data: {df.duplicated().sum()}')

Total Duplicated Data: 120


In [6]:
df = df.drop_duplicates(ignore_index = True)

In [7]:
print(f'Total Duplicated Data After Handling: {df.duplicated().sum()}')

Total Duplicated Data After Handling: 0


In [8]:
print(f'Total Missing Value: \n{df.isnull().sum()}')

Total Missing Value: 
text_emoji       0
Sentimen_Text    0
Sarcasm          0
dtype: int64


In [9]:
df['Sentimen_Text'] = df['Sentimen_Text'].replace('Positif', 1)
df['Sentimen_Text'] = df['Sentimen_Text'].replace('Negatif', 0)

df['Sarcasm'] = df['Sarcasm'].replace('Positif', 1)
df['Sarcasm'] = df['Sarcasm'].replace('Negatif', 0)

<h1>Splitting Data</h1>

In [10]:
X = df['text_emoji'].values
y = df['Sarcasm'].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

<h1>Tokenization</h1>

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

maxlen = 100
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')

tokenizer.fit_on_texts(X_train)

In [13]:
word_index = tokenizer.word_index

In [14]:
word_index

{'<OOV>': 1,
 '👍': 2,
 'luhut': 3,
 '💩': 4,
 '😁': 5,
 'indonesia': 6,
 'cak': 7,
 '😠': 8,
 'puan': 9,
 'orang': 10,
 '💯': 11,
 'nun': 12,
 'jokowi': 13,
 'presiden': 14,
 'binsar': 15,
 '👎': 16,
 'partai': 17,
 '😊': 18,
 'anies': 19,
 'rakyat': 20,
 '😭': 21,
 'maharani': 22,
 'bu': 23,
 '😨': 24,
 'ganjar': 25,
 'menteri': 26,
 'prabowo': 27,
 'pemimpin': 28,
 'semoga': 29,
 'aja': 30,
 'baswedan': 31,
 'ga': 32,
 'negara': 33,
 'masyarakat': 34,
 '😆': 35,
 'anak': 36,
 'calon': 37,
 '2024': 38,
 '💕': 39,
 'jakarta': 40,
 'kalo': 41,
 'bangsa': 42,
 'demokrasi': 43,
 'politik': 44,
 'banget': 45,
 'ketua': 46,
 'perjuangan': 47,
 'lucu': 48,
 '😔': 49,
 'mbak': 50,
 'nama': 51,
 'salah': 52,
 '✌': 53,
 'firaun': 54,
 'kerja': 55,
 'israel': 56,
 'capres': 57,
 'beliau': 58,
 'sih': 59,
 '1': 60,
 'investasi': 61,
 '😴': 62,
 'mulyadi': 63,
 'cocok': 64,
 'program': 65,
 'kadrun': 66,
 'biar': 67,
 'ri': 68,
 'dunia': 69,
 'dg': 70,
 'edi': 71,
 'lu': 72,
 'negeri': 73,
 'pilih': 74,
 'gub

<h1>Sequence</h1>

In [15]:
sequence_train = tokenizer.texts_to_sequences(X_train)
sequence_test = tokenizer.texts_to_sequences(X_test)

<h1>Padding</h1>

In [16]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train = pad_sequences(sequence_train, maxlen=100, padding='post', truncating='post')
X_test = pad_sequences(sequence_test, maxlen=100, padding='post', truncating='post')

In [17]:
X_train[0]

array([ 269,  242,  786,   19,   37,   14,  858,  284,   81,   19,  200,
        713,   17,   17,   29,   17,  465, 3800,    5,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0])

<h1>Build Matrix Embedding</h1>

In [18]:
def get_coefs(word,*arr): 
    return word, np.asarray(arr, dtype='float32')

def load_embedding(file):
    embeddings_index = dict(get_coefs(*i.split(" ")) for i in open(file, encoding='utf-8'))
    
    return embeddings_index

In [19]:
def make_embedding_matrix(embedding, tokenizer, len_voc):
    all_embs = np.stack(embedding.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    word_index = tokenizer.word_index
    # embedding_matrix = np.random.normal(emb_mean, emb_std, (len_voc, embed_size))
    embedding_matrix = np.zeros((len_voc, embed_size))
    
    for word, i in word_index.items():
        if i >= len_voc:
            continue
        embedding_vector = embedding.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [20]:
glove = load_embedding('Misc/Embeddings/embeddings100D.txt')

In [21]:
embed_matrix = make_embedding_matrix(glove, tokenizer, len(word_index))

C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3378: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [22]:
embed_matrix.shape

(6519, 100)

<h1>Implement SMOTE</h1>

In [23]:
smote = SMOTE(random_state=42)

X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [24]:
unique_values, value_counts = np.unique(y_train_resampled, return_counts=True)

In [25]:
for value, count in zip(unique_values, value_counts):
    print(f"{value}: {count}")

0: 2012
1: 2012


<h1>Create Bi-LSTM Architecture</h1>

In [26]:
model_bilstm = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(10000, 100, input_length=100),
        tf.keras.layers.Bidirectional(LSTM(100, dropout=0,recurrent_dropout=0)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
model_bilstm.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_bilstm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          1000000   
                                                                 
 bidirectional (Bidirectiona  (None, 200)              160800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 64)                12864     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,173,729
Trainable params: 1,173,729
Non-trainable params: 0
______________________________________________

<h2>Train Model Bi-LSTM</h2>

In [27]:
# Callbacks
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>=0.8): #logs.get('val_accuracy')>0.8
      print("\nAkurasi telah mencapai > 75%!")
      self.model.stop_training = True
callbacks = myCallback()

In [28]:
history_bilstm = model_bilstm.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 13s - loss: 0.6791 - accuracy: 0.5589 - val_loss: 0.6282 - val_accuracy: 0.6856 - 13s/epoch - 104ms/step
Epoch 2/10
126/126 - 9s - loss: 0.5223 - accuracy: 0.7510 - val_loss: 0.7207 - val_accuracy: 0.6492 - 9s/epoch - 68ms/step
Epoch 3/10
126/126 - 9s - loss: 0.3236 - accuracy: 0.8588 - val_loss: 0.8901 - val_accuracy: 0.6128 - 9s/epoch - 69ms/step
Epoch 4/10
126/126 - 9s - loss: 0.1799 - accuracy: 0.9299 - val_loss: 1.1493 - val_accuracy: 0.6469 - 9s/epoch - 69ms/step
Epoch 5/10
126/126 - 9s - loss: 0.1072 - accuracy: 0.9583 - val_loss: 1.3334 - val_accuracy: 0.6720 - 9s/epoch - 68ms/step
Epoch 6/10
126/126 - 9s - loss: 0.0453 - accuracy: 0.9851 - val_loss: 1.8363 - val_accuracy: 0.6970 - 9s/epoch - 69ms/step
Epoch 7/10
126/126 - 9s - loss: 0.0190 - accuracy: 0.9945 - val_loss: 2.2027 - val_accuracy: 0.7039 - 9s/epoch - 69ms/step
Epoch 8/10
126/126 - 9s - loss: 0.0299 - accuracy: 0.9925 - val_loss: 1.8093 - val_accuracy: 0.6879 - 9s/epoch - 69ms/step
Epoch 9/10
12

<h1>Create LSTM Architecture</h1>

In [29]:
model_lstm = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(10000, 100, input_length=100),
        tf.keras.layers.LSTM(100),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# Create Optimizer
model_lstm.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
)

model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          1000000   
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                6464      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,086,929
Trainable params: 1,086,929
Non-trainable params: 0
_________________________________________________________________


<h2>Train Model LSTM</h2>

In [30]:
history_lstm = model_lstm.fit(
        X_train_resampled,
        y_train_resampled,
        epochs=10,
        validation_data=(X_test, y_test),
        verbose=2
)

Epoch 1/10
126/126 - 9s - loss: 0.6941 - accuracy: 0.4973 - val_loss: 0.6925 - val_accuracy: 0.5285 - 9s/epoch - 70ms/step
Epoch 2/10
126/126 - 7s - loss: 0.6940 - accuracy: 0.4801 - val_loss: 0.6931 - val_accuracy: 0.5285 - 7s/epoch - 52ms/step
Epoch 3/10
126/126 - 6s - loss: 0.6934 - accuracy: 0.5002 - val_loss: 0.6935 - val_accuracy: 0.4715 - 6s/epoch - 52ms/step
Epoch 4/10
126/126 - 7s - loss: 0.6933 - accuracy: 0.4925 - val_loss: 0.6929 - val_accuracy: 0.5285 - 7s/epoch - 52ms/step
Epoch 5/10
126/126 - 6s - loss: 0.6934 - accuracy: 0.4901 - val_loss: 0.6940 - val_accuracy: 0.4715 - 6s/epoch - 51ms/step
Epoch 6/10
126/126 - 6s - loss: 0.6938 - accuracy: 0.4881 - val_loss: 0.6932 - val_accuracy: 0.4715 - 6s/epoch - 51ms/step
Epoch 7/10
126/126 - 6s - loss: 0.6933 - accuracy: 0.4975 - val_loss: 0.6928 - val_accuracy: 0.5285 - 6s/epoch - 51ms/step
Epoch 8/10
126/126 - 6s - loss: 0.6934 - accuracy: 0.4943 - val_loss: 0.6929 - val_accuracy: 0.5285 - 6s/epoch - 51ms/step
Epoch 9/10
126/1